<a href="https://colab.research.google.com/github/KimJisanER/code_jjambbong/blob/main/AttentiveFP_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rdkit
!pip install  dgl -f https://data.dgl.ai/wheels/cu121/repo.html
!pip install dgllife

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 48.6 MB/s eta 0:00:00
Looking in links: https://data.dgl.ai/wheels/cu121/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.0/926.0 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 5.8 MB/s eta 0:00:00


In [2]:
import matplotlib.pyplot as plt
import os
from rdkit import Chem
from rdkit.Chem import rdmolops, rdmolfiles
from rdkit import RDPaths

import dgl
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
# from dgl import model_zoo
from dgllife.model import AttentiveFPGNN
from dgllife.model import AttentiveFPReadout
from dgllife.model import AttentiveFPPredictor
from dgllife.utils import mol_to_complete_graph, mol_to_bigraph
from dgllife.utils import atom_type_one_hot
from dgllife.utils import atom_degree_one_hot
from dgllife.utils import atom_formal_charge
from dgllife.utils import atom_num_radical_electrons
from dgllife.utils import atom_hybridization_one_hot
from dgllife.utils import atom_total_num_H_one_hot
from dgllife.utils import one_hot_encoding
from dgllife.utils import CanonicalAtomFeaturizer
from dgllife.utils import CanonicalBondFeaturizer
from dgllife.utils import ConcatFeaturizer
from dgllife.utils import BaseAtomFeaturizer
from dgllife.utils import BaseBondFeaturizer
from dgllife.utils import one_hot_encoding
from dgl.data.utils import split_dataset

from functools import partial
from sklearn.metrics import roc_auc_score

In [3]:
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
import rdkit

In [4]:
def load_sdf_file(file_path):
    suppl = Chem.SDMolSupplier(file_path)
    mols = [x for x in suppl if x is not None]
    df = PandasTools.LoadSDF(file_path)
    df['Molecule'] = mols
    # df = df[['chembl_id', 'Molecule']]
    df = df.dropna()
    return df

In [5]:
sdf_file_path = '/content/drive/MyDrive/Class_A_0115.sdf'

df = load_sdf_file(sdf_file_path)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 188435 entries, 0 to 188435
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   chembl_id           188435 non-null  object
 1   chembl_pref_name    188435 non-null  object
 2   target_chembl_id    188435 non-null  object
 3   target_accession    188435 non-null  object
 4   mw_freebase         188435 non-null  object
 5   alogp               188435 non-null  object
 6   bioactivities       188435 non-null  object
 7   pChEMBL_value_Ki    188435 non-null  object
 8   pChEMBL_value_Kd    188435 non-null  object
 9   pChEMBL_value_IC50  188435 non-null  object
 10  pChEMBL_value_EC50  188435 non-null  object
 11  ID                  188435 non-null  object
 12  ROMol               188435 non-null  object
 13  Molecule            188435 non-null  object
dtypes: object(14)
memory usage: 21.6+ MB


In [7]:
df[['mw_freebase', 'alogp', 'pChEMBL_value_Ki', 'pChEMBL_value_Kd', 'pChEMBL_value_IC50', 'pChEMBL_value_EC50']] = df[['mw_freebase', 'alogp', 'pChEMBL_value_Ki', 'pChEMBL_value_Kd', 'pChEMBL_value_IC50', 'pChEMBL_value_EC50']].replace('None',None)
df_Ki = df.dropna(subset=['pChEMBL_value_Ki'])
df_Ki=df_Ki[['chembl_id','target_accession','pChEMBL_value_Ki', 'Molecule']]

In [8]:
df_Ki

,chembl_id,target_accession,pChEMBL_value_Ki,Molecule
5,CHEMBL104593,P19327,9.44,<rdkit.Chem.rdchem.Mol object at 0x78bc642eb680>
6,CHEMBL105926,P19327,6.83,<rdkit.Chem.rdchem.Mol object at 0x78bc64314040>
7,CHEMBL108026,P19327,7.46,<rdkit.Chem.rdchem.Mol object at 0x78bc64314200>
9,CHEMBL1086958,P19327,8.21,<rdkit.Chem.rdchem.Mol object at 0x78bc643142e0>
10,CHEMBL1086959,P19327,6.85,<rdkit.Chem.rdchem.Mol object at 0x78bc64314350>
...,...,...,...,...
187547,CHEMBL410333,Q9Z2J6,6.82,<rdkit.Chem.rdchem.Mol object at 0x78bc626317e0>
187548,CHEMBL411315,Q9Z2J6,7.48,<rdkit.Chem.rdchem.Mol object at 0x78bc62631850>
187549,CHEMBL563949,Q9Z2J6,7.55,<rdkit.Chem.rdchem.Mol object at 0x78bc626318c0>
188379,CHEMBL569766,Q99527,8.15,<rdkit.Chem.rdchem.Mol object at 0x78bc62648740>


In [9]:
import pandas as pd

fasta_file = '/content/drive/MyDrive/class_A_bindingsite.fasta'

with open(fasta_file, 'r') as f:
    lines = f.readlines()

seqs = []
headers = []
header_positions = []  # Store the positions of headers satisfying the condition
for i, line in enumerate(lines):
    if line.startswith('>'):
        headers.append(line.strip().upper())
        seqs.append('')
        if '>SP' in line and 'HUMAN' in line and 'HTR' in line:
            header_positions.append(i)  # Store the position of the header
    else:
        seqs[-1] += line.strip()

df_2 = pd.DataFrame(columns=['accession', 'sequence'])
data = []

for header, seq in zip(headers, seqs):
    accession = header.split('|')[1]
    data.append([accession, seq])

df_2 = pd.DataFrame(data, columns=['accession', 'sequence'])

In [10]:
def embed_sequence(sequence):

    blosum62_matrix = {
        'A': [4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1, -1, -2, -1, 1, 0, -3, -2, 0],
        'R': [-1, 5, 0, -2, -3, 1, 0, -2, 0, -3, -2, 2, -1, -3, -2, -1, -1, -3, -2, -3],
        'N': [-2, 0, 6, 1, -3, 0, 0, 0, 1, -3, -3, 0, -2, -3, -2, 1, 0, -4, -2, -3],
        'D': [-2, -2, 1, 6, -3, 0, 2, -1, -1, -3, -4, -1, -3, -3, -1, 0, -1, -4, -3, -3],
        'C': [0, -3, -3, -3, 9, -3, -4, -3, -3, -1, -1, -3, -1, -2, -3, -1, -1, -2, -2, -1],
        'Q': [-1, 1, 0, 0, -3, 5, 2, -2, 0, -3, -2, 1, 0, -3, -1, 0, -1, -2, -1, -2],
        'E': [-1, 0, 0, 2, -4, 2, 5, -2, 0, -3, -3, 1, -2, -3, -1, 0, -1, -3, -2, -2],
        'G': [0, -2, 0, -1, -3, -2, -2, 6, -2, -4, -4, -2, -3, -3, -2, 0, -2, -2, -3, -3],
        'H': [-2, 0, 1, -1, -3, 0, 0, -2, 8, -3, -3, -1, -2, -1, -2, -1, -2, -2, 2, -3],
        'I': [-1, -3, -3, -3, -1, -3, -3, -4, -3, 4, 2, -3, 1, 0, -3, -2, -1, -3, -1, 3],
        'L': [-1, -2, -3, -4, -1, -2, -3, -4, -3, 2, 4, -2, 2, 0, -3, -2, -1, -2, -1, 1],
        'K': [-1, 2, 0, -1, -3, 1, 1, -2, -1, -3, -2, 5, -1, -3, -1, 0, -1, -3, -2, -2],
        'M': [-1, -1, -2, -3, -1, 0, -2, -3, -2, 1, 2, -1, 5, 0, -2, -1, -1, -1, -1, 1],
        'F': [-2, -3, -3, -3, -2, -3, -3, -3, -1, 0, 0, -3, 0, 6, -4, -2, -2, 1, 3, -1],
        'P': [-1, -2, -2, -1, -3, -1, -1, -2, -2, -3, -3, -1, -2, -4, 7, -1, -1, -4, -3, -2],
        'S': [1, -1, 1, 0, -1, 0, 0, 0, -1, -2, -2, 0, -1, -2, -1, 4, 1, -3, -2, -2],
        'T': [0, -1, 0, -1, -1, -1, -1, -2, -2, -1, -1, -1, -1, -2, -1, 1, 5, -2, -2, 0],
        'W': [-3, -3, -4, -4, -2, -2, -3, -2, -2, -3, -2, -3, -1, 1, -4, -3, -2, 11, 2, -3],
        'Y': [-2, -2, -2, -3, -2, -1, -2, -3, 2, -1, -1, -2, -1, 3, -3, -2, -2, 2, 7, -1],
        'V': [0, -3, -3, -3, -1, -2, -2, -3, -3, 3, 1, -2, 1, -1, -2, -2, 0, -3, -1, 4],
        'B': [-2, -1, 3, 4, -3, 0, 1, -1, 0, -3, -4, 0, -3, -3, -2, 0, -1, -4, -3, -3],
        'Z': [-1, 0, 0, 1, -3, 3, 4, -2, 0, -3, -3, 1, -1, -3, -1, 0, -1, -3, -2, -2],
        'X': [0, -1, -1, -1, -2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -2, 0, 0, -2, -1, -1],
        '-': [-4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4],
        '_': [-4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4],
        '*': [-4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4]
    }

    embedding = np.zeros((len(sequence), len(blosum62_matrix['A'])))
    for i, amino_acid in enumerate(sequence):
        if amino_acid not in blosum62_matrix:
            raise ValueError("Invalid amino acid: " + amino_acid)
        embedding[i] = blosum62_matrix[amino_acid]

    return embedding

In [11]:
df_2['sequence'] = np.transpose(df_2['sequence'].apply(embed_sequence) + 5)/16

In [12]:
df_2

,accession,sequence
0,A0A2K5XZ43,"[[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.06..."
1,A0A673UJF2,"[[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.06..."
2,A6I5H2,"[[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.06..."
3,A0A3Q7QQR7,"[[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.06..."
4,A0A2Y9LBV4,"[[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.06..."
...,...,...
22051,Q96RI9,"[[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.06..."
22052,G1SUU9,"[[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.06..."
22053,A0A6P6I081,"[[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.06..."
22054,A0A8C6HEP6,"[[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.06..."


In [13]:
data = pd.merge(df_Ki, df_2, left_on = 'target_accession', right_on = 'accession' ).drop(columns='target_accession')

# data.to_pickle("/content/drive/MyDrive/Class_A_df_0229")

In [14]:
data

,chembl_id,pChEMBL_value_Ki,Molecule,accession,sequence
0,CHEMBL104593,9.44,<rdkit.Chem.rdchem.Mol object at 0x78bc642eb680>,P19327,"[[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.06..."
1,CHEMBL105926,6.83,<rdkit.Chem.rdchem.Mol object at 0x78bc64314040>,P19327,"[[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.06..."
2,CHEMBL108026,7.46,<rdkit.Chem.rdchem.Mol object at 0x78bc64314200>,P19327,"[[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.06..."
3,CHEMBL1086958,8.21,<rdkit.Chem.rdchem.Mol object at 0x78bc643142e0>,P19327,"[[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.06..."
4,CHEMBL1086959,6.85,<rdkit.Chem.rdchem.Mol object at 0x78bc64314350>,P19327,"[[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.06..."
...,...,...,...,...,...
114525,CHEMBL67464,7.68,<rdkit.Chem.rdchem.Mol object at 0x78bc625e2ea0>,P47936,"[[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.06..."
114526,CHEMBL68641,8.72,<rdkit.Chem.rdchem.Mol object at 0x78bc625e2f10>,P47936,"[[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.06..."
114527,CHEMBL947,8.74,<rdkit.Chem.rdchem.Mol object at 0x78bc625e2f80>,P47936,"[[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.06..."
114528,CHEMBL569766,8.15,<rdkit.Chem.rdchem.Mol object at 0x78bc62648740>,Q99527,"[[0.25, 0.3125, 0.3125, 0.4375, 0.0625, 0.4375..."


In [29]:
data['accession'].value_counts()

P14416    4917
P29274    4229
P30542    3726
P35462    3526
P34972    3436
          ... 
Q9R1C8       1
P32304       1
P32247       1
O54798       1
Q8K418       1
Name: accession, Length: 290, dtype: int64

In [30]:
data = data[data['accession']=='P14416']

In [31]:
data

,chembl_id,pChEMBL_value_Ki,Molecule,accession,sequence
26069,CHEMBL102414,5.43,<rdkit.Chem.rdchem.Mol object at 0x78bc63b70c80>,P14416,"[[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.06..."
26070,CHEMBL102582,8.21,<rdkit.Chem.rdchem.Mol object at 0x78bc63b70cf0>,P14416,"[[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.06..."
26071,CHEMBL102632,6.20,<rdkit.Chem.rdchem.Mol object at 0x78bc63b70d60>,P14416,"[[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.06..."
26072,CHEMBL103441,5.97,<rdkit.Chem.rdchem.Mol object at 0x78bc63b70dd0>,P14416,"[[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.06..."
26073,CHEMBL1036,7.04,<rdkit.Chem.rdchem.Mol object at 0x78bc63b70e40>,P14416,"[[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.06..."
...,...,...,...,...,...
30981,CHEMBL98946,6.38,<rdkit.Chem.rdchem.Mol object at 0x78bc63a0f450>,P14416,"[[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.06..."
30982,CHEMBL99198,7.60,<rdkit.Chem.rdchem.Mol object at 0x78bc63a0f530>,P14416,"[[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.06..."
30983,CHEMBL99208,6.40,<rdkit.Chem.rdchem.Mol object at 0x78bc63a0f5a0>,P14416,"[[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.06..."
30984,CHEMBL99355,6.71,<rdkit.Chem.rdchem.Mol object at 0x78bc63a0f610>,P14416,"[[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.06..."


In [32]:
class AttentiveFP_module(nn.Module):
    """AttentiveFP for regression and classification on graphs.

    AttentiveFP is introduced in
    `Pushing the Boundaries of Molecular Representation for Drug Discovery with the Graph
    Attention Mechanism. <https://www.ncbi.nlm.nih.gov/pubmed/31408336>`__

    Parameters
    ----------
    node_feat_size : int
        Size for the input node features.
    edge_feat_size : int
        Size for the input edge features.
    num_layers : int
        Number of GNN layers. Default to 2.
    num_timesteps : int
        Times of updating the graph representations with GRU. Default to 2.
    graph_feat_size : int
        Size for the learned graph representations. Default to 200.
    n_tasks : int
        Number of tasks, which is also the output size. Default to 1.
    dropout : float
        Probability for performing the dropout. Default to 0.
    """
    def __init__(self,
                 node_feat_size,
                 edge_feat_size,
                 num_layers=2,
                 num_timesteps=2,
                 graph_feat_size=200,
                 n_tasks=1,
                 dropout=0.):
        super(AttentiveFPPredictor, self).__init__()

        self.gnn = AttentiveFPGNN(node_feat_size=node_feat_size,
                                  edge_feat_size=edge_feat_size,
                                  num_layers=num_layers,
                                  graph_feat_size=graph_feat_size,
                                  dropout=dropout)
        self.readout = AttentiveFPReadout(feat_size=graph_feat_size,
                                          num_timesteps=num_timesteps,
                                          dropout=dropout)
        self.predict = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(graph_feat_size, n_tasks)
        )

    def forward(self, g, node_feats, edge_feats, get_node_weight=False):
        """Graph-level regression/soft classification.

        Parameters
        ----------
        g : DGLGraph
            DGLGraph for a batch of graphs.
        node_feats : float32 tensor of shape (V, node_feat_size)
            Input node features. V for the number of nodes.
        edge_feats : float32 tensor of shape (E, edge_feat_size)
            Input edge features. E for the number of edges.
        get_node_weight : bool
            Whether to get the weights of atoms during readout. Default to False.

        Returns
        -------
        float32 tensor of shape (G, n_tasks)
            Prediction for the graphs in the batch. G for the number of graphs.
        node_weights : list of float32 tensor of shape (V, 1), optional
            This is returned when ``get_node_weight`` is ``True``.
            The list has a length ``num_timesteps`` and ``node_weights[i]``
            gives the node weights in the i-th update.
        """
        node_feats = self.gnn(g, node_feats, edge_feats)
        if get_node_weight:
            g_feats, node_weights = self.readout(g, node_feats, get_node_weight)
            return self.predict(g_feats), node_weights
        else:
            g_feats = self.readout(g, node_feats, get_node_weight)
            return g_feats

In [33]:
def chirality(atom):
    try:
        return one_hot_encoding(atom.GetProp('_CIPCode'), ['R', 'S']) + \
               [atom.HasProp('_ChiralityPossible')]
    except:
        return [False, False] + [atom.HasProp('_ChiralityPossible')]

def collate_molgraphs(data):
    """Batching a list of datapoints for dataloader.
    Parameters
    ----------
    data : list of 3-tuples or 4-tuples.
        Each tuple is for a single datapoint, consisting of
        a SMILES, a DGLGraph, all-task labels and optionally
        a binary mask indicating the existence of labels.
    Returns
    -------
    smiles : list
        List of smiles
    bg : BatchedDGLGraph
        Batched DGLGraphs
    labels : Tensor of dtype float32 and shape (B, T)
        Batched datapoint labels. B is len(data) and
        T is the number of total tasks.
    masks : Tensor of dtype float32 and shape (B, T)
        Batched datapoint binary mask, indicating the
        existence of labels. If binary masks are not
        provided, return a tensor with ones.
    """
    assert len(data[0]) in [3, 4], \
        'Expect the tuple to be of length 3 or 4, got {:d}'.format(len(data[0]))
    if len(data[0]) == 3:
        smiles, graphs, labels = map(list, zip(*data))
        masks = None
    else:
        smiles, graphs, labels, masks = map(list, zip(*data))

    bg = dgl.batch(graphs)
    bg.set_n_initializer(dgl.init.zero_initializer)
    bg.set_e_initializer(dgl.init.zero_initializer)
    labels = torch.stack(labels, dim=0)

    if masks is None:
        masks = torch.ones(labels.shape)
    else:
        masks = torch.stack(masks, dim=0)
    return smiles, bg, labels, masks

atom_featurizer = BaseAtomFeaturizer(
                 {'hv': ConcatFeaturizer([
                  partial(atom_type_one_hot, allowable_set=[
                          'B', 'C', 'N', 'O', 'F', 'Si', 'P', 'S', 'Cl', 'As', 'Se', 'Br', 'Te', 'I', 'At'],
                    encode_unknown=True),
                  partial(atom_degree_one_hot, allowable_set=list(range(6))),
                  atom_formal_charge, atom_num_radical_electrons,
                  partial(atom_hybridization_one_hot, encode_unknown=True),
                  lambda atom: [0], # A placeholder for aromatic information,
                    atom_total_num_H_one_hot, chirality
                 ],
                )})
bond_featurizer = BaseBondFeaturizer({
                                     'he': lambda bond: [0 for _ in range(10)]
    })


In [34]:
def embed_sequence(sequence):

    blosum62_matrix = {
        'A': [4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1, -1, -2, -1, 1, 0, -3, -2, 0],
        'R': [-1, 5, 0, -2, -3, 1, 0, -2, 0, -3, -2, 2, -1, -3, -2, -1, -1, -3, -2, -3],
        'N': [-2, 0, 6, 1, -3, 0, 0, 0, 1, -3, -3, 0, -2, -3, -2, 1, 0, -4, -2, -3],
        'D': [-2, -2, 1, 6, -3, 0, 2, -1, -1, -3, -4, -1, -3, -3, -1, 0, -1, -4, -3, -3],
        'C': [0, -3, -3, -3, 9, -3, -4, -3, -3, -1, -1, -3, -1, -2, -3, -1, -1, -2, -2, -1],
        'Q': [-1, 1, 0, 0, -3, 5, 2, -2, 0, -3, -2, 1, 0, -3, -1, 0, -1, -2, -1, -2],
        'E': [-1, 0, 0, 2, -4, 2, 5, -2, 0, -3, -3, 1, -2, -3, -1, 0, -1, -3, -2, -2],
        'G': [0, -2, 0, -1, -3, -2, -2, 6, -2, -4, -4, -2, -3, -3, -2, 0, -2, -2, -3, -3],
        'H': [-2, 0, 1, -1, -3, 0, 0, -2, 8, -3, -3, -1, -2, -1, -2, -1, -2, -2, 2, -3],
        'I': [-1, -3, -3, -3, -1, -3, -3, -4, -3, 4, 2, -3, 1, 0, -3, -2, -1, -3, -1, 3],
        'L': [-1, -2, -3, -4, -1, -2, -3, -4, -3, 2, 4, -2, 2, 0, -3, -2, -1, -2, -1, 1],
        'K': [-1, 2, 0, -1, -3, 1, 1, -2, -1, -3, -2, 5, -1, -3, -1, 0, -1, -3, -2, -2],
        'M': [-1, -1, -2, -3, -1, 0, -2, -3, -2, 1, 2, -1, 5, 0, -2, -1, -1, -1, -1, 1],
        'F': [-2, -3, -3, -3, -2, -3, -3, -3, -1, 0, 0, -3, 0, 6, -4, -2, -2, 1, 3, -1],
        'P': [-1, -2, -2, -1, -3, -1, -1, -2, -2, -3, -3, -1, -2, -4, 7, -1, -1, -4, -3, -2],
        'S': [1, -1, 1, 0, -1, 0, 0, 0, -1, -2, -2, 0, -1, -2, -1, 4, 1, -3, -2, -2],
        'T': [0, -1, 0, -1, -1, -1, -1, -2, -2, -1, -1, -1, -1, -2, -1, 1, 5, -2, -2, 0],
        'W': [-3, -3, -4, -4, -2, -2, -3, -2, -2, -3, -2, -3, -1, 1, -4, -3, -2, 11, 2, -3],
        'Y': [-2, -2, -2, -3, -2, -1, -2, -3, 2, -1, -1, -2, -1, 3, -3, -2, -2, 2, 7, -1],
        'V': [0, -3, -3, -3, -1, -2, -2, -3, -3, 3, 1, -2, 1, -1, -2, -2, 0, -3, -1, 4],
        'B': [-2, -1, 3, 4, -3, 0, 1, -1, 0, -3, -4, 0, -3, -3, -2, 0, -1, -4, -3, -3],
        'Z': [-1, 0, 0, 1, -3, 3, 4, -2, 0, -3, -3, 1, -1, -3, -1, 0, -1, -3, -2, -2],
        'X': [0, -1, -1, -1, -2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -2, 0, 0, -2, -1, -1],
        '-': [-4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4],
        '_': [-4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4],
        '*': [-4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4]
    }

    embedding = np.zeros((len(sequence), len(blosum62_matrix['A'])))
    for i, amino_acid in enumerate(sequence):
        if amino_acid not in blosum62_matrix:
            raise ValueError("Invalid amino acid: " + amino_acid)
        embedding[i] = blosum62_matrix[amino_acid]

    return embedding

In [35]:
from sklearn.model_selection import train_test_split

# X = data.drop(columns='pChEMBL_value_Ki')
# y = data['pChEMBL_value_Ki'].astype('float32')

train, test = train_test_split(data, test_size=0.1, random_state=42)
train, val = train_test_split(train, test_size=0.1, random_state=42)

In [36]:
import torch

# CUDA가 사용 가능한지 확인
if torch.cuda.is_available():
    # CUDA 장치의 개수 확인
    print(f"CUDA device count: {torch.cuda.device_count()}")
    # 각 CUDA 장치의 이름과 CUDA 버전 확인
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}, CUDA version: {torch.version.cuda}")
else:
    print("CUDA is not available.")


CUDA device count: 1
Device 0: Tesla V100-SXM2-16GB, CUDA version: 12.1


In [40]:
# train=os.path.join(RDPaths.RDDocsDir, 'Book/data/solubility.train.sdf')
# test=os.path.join(RDPaths.RDDocsDir, 'Book/data/solubility.test.sdf')

train_mols = train['Molecule']
train_smi =[Chem.MolToSmiles(m) for m in train_mols]
train_sol = torch.tensor([float(mol.GetProp('pChEMBL_value_Ki')) for mol in train_mols]).reshape(-1,1)

test_mols =  test['Molecule']
test_smi = [Chem.MolToSmiles(m) for m in test_mols]
test_sol = torch.tensor([float(mol.GetProp('pChEMBL_value_Ki')) for mol in test_mols]).reshape(-1,1)

val_mols =  val['Molecule']
val_smi = [Chem.MolToSmiles(m) for m in val_mols]
val_sol = torch.tensor([float(mol.GetProp('pChEMBL_value_Ki')) for mol in val_mols]).reshape(-1,1)

train_graph =[mol_to_bigraph(mol,
                           node_featurizer=atom_featurizer,
                           edge_featurizer=bond_featurizer) for mol in train_mols]

test_graph =[mol_to_bigraph(mol,
                           node_featurizer=atom_featurizer,
                           edge_featurizer=bond_featurizer) for mol in test_mols]

val_graph =[mol_to_bigraph(mol,
                           node_featurizer=atom_featurizer,
                           edge_featurizer=bond_featurizer) for mol in val_mols]

In [42]:
def run_a_train_epoch(n_epochs, epoch, model, data_loader,loss_criterion, optimizer):
    model.train()
    total_loss = 0
    losses = []

    for batch_id, batch_data in enumerate(data_loader):
        batch_data
        smiles, bg, labels, masks = batch_data
        if torch.cuda.is_available():

          bg = bg.to('cuda:0')
          labels = labels.to('cuda:0')
          masks = masks.to('cuda:0')

        prediction = model(bg, bg.ndata['hv'], bg.edata['he'])
        loss = (loss_criterion(prediction, labels)*(masks != 0).float()).mean()
        #loss = loss_criterion(prediction, labels)
        #print(loss.shape)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.data.item())

    #total_score = np.mean(train_meter.compute_metric('rmse'))
    total_score = np.mean(losses)
    print('epoch {:d}/{:d}, training {:.4f}'.format( epoch + 1, n_epochs,  total_score))
    return total_score

def run_a_validation_epoch(model, data_loader, loss_criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch_id, batch_data in enumerate(data_loader):
            smiles, bg, labels, masks = batch_data
            if torch.cuda.is_available():
                bg = bg.to('cuda:0')
                labels = labels.to('cuda:0')
                masks = masks.to('cuda:0')

            prediction = model(bg, bg.ndata['hv'], bg.edata['he'])
            loss = (loss_criterion(prediction, labels)*(masks != 0).float()).mean()
            total_loss += loss.item()

    total_score = total_loss / len(data_loader)
    print('Validation loss: {:.4f}'.format(total_score))
    return total_score

In [44]:

model = AttentiveFPPredictor(node_feat_size=39,
                                  edge_feat_size=10,
                                  num_layers=2,
                                  num_timesteps=2,
                                  graph_feat_size=200,
                                  n_tasks=1,
                                  dropout=0.2)
model = model.to('cuda:0')

train_loader = DataLoader(dataset=list(zip(train_smi, train_graph, train_sol)), batch_size=128, collate_fn=collate_molgraphs)
test_loader = DataLoader(dataset=list(zip(test_smi, test_graph, test_sol)), batch_size=128, collate_fn=collate_molgraphs)
val_loader = DataLoader(dataset=list(zip(val_smi, val_graph, val_sol)), batch_size=128, collate_fn=collate_molgraphs)

loss_fn = nn.MSELoss(reduction='none')
optimizer = torch.optim.Adam(model.parameters(), lr=10 ** (-2.5), weight_decay=10 ** (-5.0),)
n_epochs = 300
epochs = []
scores = []
for e in range(n_epochs):
    score = run_a_train_epoch(n_epochs, e, model, train_loader, loss_fn, optimizer)
    val_score = run_a_validation_epoch(model, val_loader, loss_fn)
    epochs.append(e)
    scores.append(score)
model.eval()

epoch 1/300, training 8.6255
Validation loss: 1.0690
epoch 2/300, training 1.2884
Validation loss: 1.0284
epoch 3/300, training 1.2152
Validation loss: 1.0307
epoch 4/300, training 1.1900
Validation loss: 1.0208
epoch 5/300, training 1.1543
Validation loss: 0.9929
epoch 6/300, training 1.1079
Validation loss: 1.0151
epoch 7/300, training 1.1061
Validation loss: 0.9346
epoch 8/300, training 1.0931
Validation loss: 0.9666
epoch 9/300, training 1.0615
Validation loss: 0.9265
epoch 10/300, training 1.0416
Validation loss: 0.9152
epoch 11/300, training 1.0197
Validation loss: 0.8798
epoch 12/300, training 1.0345
Validation loss: 0.8737
epoch 13/300, training 1.0049
Validation loss: 0.8607
epoch 14/300, training 0.9760
Validation loss: 0.8535
epoch 15/300, training 0.9707
Validation loss: 0.8367
epoch 16/300, training 0.9326
Validation loss: 0.8072
epoch 17/300, training 0.8969
Validation loss: 0.7818
epoch 18/300, training 0.8833
Validation loss: 0.7897
epoch 19/300, training 0.8736
Validat

AttentiveFPPredictor(
  (gnn): AttentiveFPGNN(
    (init_context): GetContext(
      (project_node): Sequential(
        (0): Linear(in_features=39, out_features=200, bias=True)
        (1): LeakyReLU(negative_slope=0.01)
      )
      (project_edge1): Sequential(
        (0): Linear(in_features=49, out_features=200, bias=True)
        (1): LeakyReLU(negative_slope=0.01)
      )
      (project_edge2): Sequential(
        (0): Dropout(p=0.2, inplace=False)
        (1): Linear(in_features=400, out_features=1, bias=True)
        (2): LeakyReLU(negative_slope=0.01)
      )
      (attentive_gru): AttentiveGRU1(
        (edge_transform): Sequential(
          (0): Dropout(p=0.2, inplace=False)
          (1): Linear(in_features=200, out_features=200, bias=True)
        )
        (gru): GRUCell(200, 200)
      )
    )
    (gnn_layers): ModuleList(
      (0): GNNLayer(
        (project_edge): Sequential(
          (0): Dropout(p=0.2, inplace=False)
          (1): Linear(in_features=400, out_fea